<div class="alert alert-block alert-info" style="outline-style: solid;">
<h1>1.0 Import Modules</h1>
</div>

First, you need to import the following modules. Make sure the sys.path.append('') points to the location on your computer that you saved pyce_tools.

In [2]:
import pandas as pd
import os
import datetime
sys.path.append('C:\\Users\\trueblood\\projects\\pyce_tools')
import pyce_tools.pyce_tools as pt

Cool party!


For this tutorial, we'll take a raw data file of aerosol sample type and preprocess it, then do some basic calculations and analysis. 

The first step is to pre-process all the data.

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h1>2.0 Pre-processing of INP Data</h1>
</div>

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h2>2.1 Raw Data Calculation</h2>
</div>

Before we do anything, we need to calculate the raw blank data into a calculated report file.

In [3]:
# Create blank file
issues = None
type_ = 'aerosol'
process = 'uf'
collection_date = '06042020 '
sample_name = 'Bubbler 3 stage BLANK pc'
analysis_date = '01/01/21'
num_tubes = 26
vol_tube = 0.2
rinse_vol = 20
size = 'sub'
location = 'bubbler'
raw = pt.calculate_raw_blank(type_, process, location, sample_name, collection_date,
                  analysis_date, issues, num_tubes, vol_tube, rinse_vol, size)

...Raw blank data calculated!


Now that we've created our blank file, we can use it to calculate a raw data file from a sample. Note that we pass the file into the 'blank_source' parameter below. Pyce Tools uses the metadata that you pass when calling it to calculate the report files, so it's important you fill everything in.

In [4]:
# Calculate aerosol sample
issues = None
blank_source = '..\\data\\interim\\IN\\calculated\\blank\\bubbler_blank_uf_sub_060420_calculated.xlsx'
type_ = 'aerosol'
size = 'sub'
location = 'bubbler'
process = 'uf'
sample_name = 'Day 01 bubbler 3 stage PC'
collection_date = '17032020 11h25'
sample_stop_time = '18032020 09h45'
analysis_date = '01/01/2021'
num_tubes = 26
vol_tube = 0.2
flow_start = 10.50
flow_stop = 9.35
rinse_vol = 20

raw = pt.calculate_raw(blank_source, type_, location, process, sample_name, collection_date, analysis_date, issues, num_tubes, vol_tube, rinse_vol, size, flow_start, flow_stop, sample_stop_time)

...IN data calculated!
Calculated report file saved to ..\data\interim\IN\calculated\aerosol\bubbler\aerosol_bubbler_uf_sub_170320_1125_calculated.xlsx.


Running the above code will create a calculated report file in which the blank corrected values are given. Take a minute to open the report file now (file path is given above) and have a look yourself. Also be sure to check the 'tubeholderfilling' tab which gives an overview of all the cells. Sometimes unexpected things happen during analysis, like the LED turns off mid experiment. You should compare this with the final image from the LINDA to ensure the automated process correctly identified frozen vs nonfrozen cells. 

For example, during this experiment, the LED turned off several times. This caused the program to assume all cells froze. Go into the 'freezepointdetection' tab. Whenever you see several rows where EVERY column has indicated freezing occurred, you can assume it's because the LED turned off. To fix this, simply delete those rows. 

Also note a separate tab for heated vs unheated samples is given (e.g., summary_UF_UH and summary_UF_H). The code is written to assume the bottom 26 cells are unheated and the top 26 cells are heated. You can alter this as needed. Each summary tab has calculations for blank corrected concentrations and uncertainty as well as metadata. Once you're familiar with this report file you can come back here and begin the next step.

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h2>2.2 Clean Calculated Report File</h2>
</div>


Cleaning the available calculated report files is as sample as calling clean_calculated_in() function and passing the parameters for INP type and location. The script will automatically find all relevant files in the folder and combine them into a single time series.

In [12]:
# Create cleaned files of INP and uncertainty
pt.clean_calculated_in(type_='aerosol',location='bubbler')
pt.calculate_wilson_errors(project='tutorial', location='bubbler', type_='aerosol', n=26)

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h2>2.3 Load Cleaned Data and Final Preprocessing</h2>
</div>



Load the cleaned INP data and uncertainty files. Do a final bit of preprocessing. This is unique to each experiment but I've given an example below.

In [39]:
# Load cleaned data and create time timeseries index in New Zealand Standard Time
inp_bubbler = pd.read_csv('..\\data\\interim\\IN\\cleaned\\combinedtimeseries\\aerosol\\bubbler_17032020_17032020.csv', index_col='datetime')
inp_bubbler.index = pd.to_datetime(inp_bubbler.index, format='%d%m%Y %Hh%M')
inp_bubbler= inp_bubbler.tz_localize('NZ')

bubbler_error = pd.read_csv('..\\data\\interim\\IN\\cleaned\\combinedtimeseries\\aerosol\\bubbler_170320_170320wilson_error.csv', index_col='datetime', parse_dates=True)
bubbler_error.index=pd.to_datetime(bubbler_error.index, dayfirst=True, format='%d%m%Y %Hh%M')
bubbler_error= bubbler_error.tz_localize('NZ')


In [40]:
inp_bubbler.head()

,-18.0,-17.9,-17.8,-17.7,-17.6,-17.5,-17.4,-17.3,-17.2,-17.1,...,-1.2,-1.1,-1.0,start_date,stop_date,size,process,type,location,filtered
datetime,,,,,,,,,,,,,,,,,,,,,
2020-03-17 11:25:00+13:00,0.007185,0.005814,0.005212,0.003651,0.003651,0.003651,0.003195,0.003195,0.003195,0.003195,...,0.0,0.0,0.0,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf
2020-03-17 11:25:00+13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf


In [41]:
# Create melted dataframes with INP/m^3
inp_bubbler_melt=inp_bubbler.reset_index().melt(id_vars = ['datetime','start_date','stop_date','size','process', 'type','location','filtered'], var_name = 'temp', value_name='inp/l')
inp_bubbler_melt['inp/m^3'] = inp_bubbler_melt['inp/l']*1000

bubbler_error_melt = pd.melt(bubbler_error.reset_index(), id_vars = ['datetime','time','process','filtered','location','type','value_name', 'size'], var_name = 'temp', value_name='error (inp/l)')
bubbler_error_melt=bubbler_error_melt.drop(columns='time')
bubbler_error_melt['error (inp/m^3)'] = bubbler_error_melt['error (inp/l)']*1000

In [42]:
inp_bubbler_melt

,datetime,start_date,stop_date,size,process,type,location,filtered,temp,inp/l,inp/m^3
0,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-18.0,0.007185,7.184567
1,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-18.0,NaN,NaN
2,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-17.9,0.005814,5.813676
3,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-17.9,NaN,NaN
4,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-17.8,0.005212,5.211829
...,...,...,...,...,...,...,...,...,...,...,...
337,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-1.2,0.000000,0.000000
338,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-1.1,0.000000,0.000000
339,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-1.1,0.000000,0.000000
340,2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-1.0,0.000000,0.000000


In [43]:
# create a pivoted uway error dataframe then merge with the actual data
bubbler_error_pivot = bubbler_error_melt.pivot_table(index=['process','datetime','temp','type','location','filtered'], values=['error (inp/l)'], columns='value_name')
bubbler_df = pd.merge(inp_bubbler_melt,bubbler_error_pivot.reset_index(), on=['datetime','temp','process','type','location','filtered'])

# clean column names
bubbler_df = bubbler_df.rename(columns={(
    'error (inp/l)','IN/L_lower'):'IN/L_lower',
    ('error (inp/l)','IN/L_upper'):'IN/L_upper',
    ('error (inp/l)','error_minus_y'):'error_minus_y',
    ('error (inp/l)','error_y'):'error_y'})

bubbler_df.set_index('datetime', inplace=True)

In [44]:
bubbler_df

,start_date,stop_date,size,process,type,location,filtered,temp,inp/l,inp/m^3,IN/L_lower,IN/L_upper,error_minus_y,error_y
datetime,,,,,,,,,,,,,,
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-18.0,0.007185,7.184567,0.004165,0.011240,0.003019,0.004055
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-18.0,NaN,NaN,0.015414,NaN,NaN,NaN
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-17.9,0.005814,5.813676,0.003292,0.009371,0.002521,0.003558
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-17.9,NaN,NaN,0.015414,NaN,NaN,NaN
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-17.8,0.005212,5.211829,0.002906,0.008553,0.002305,0.003342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-1.2,0.000000,0.000000,0.000000,0.001036,0.000000,0.001036
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,UH,aerosol,bubbler,uf,-1.1,0.000000,0.000000,0.000000,0.001036,0.000000,0.001036
2020-03-17 11:25:00+13:00,17032020 11h25,18032020 09h45,sub,H,aerosol,bubbler,uf,-1.1,0.000000,0.000000,0.000000,0.001036,0.000000,0.001036


<div class="alert alert-block alert-info" style="outline-style: solid;">
<h2>3.0 Analysis</h2>
</div>

Now we'll begin analysis of our INP data. To start, we'll calculate the surface area normalized INP concentrations. Some of the code below may seem complicated, but just follow along and it will make sense. In the future I'd like to make this process a bit less 'hacky'.

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h2>3.1 Calculate Surface Area Normalized INP Concentrations</h2>
</div>

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h3>3.1.1 Clean Inverted SMPS Data</h3>
</div>


In [11]:
# Define constants
instr= 'scanotron'

# Note that we have placed the inverted data in the 'interim' folder. This is because technically the raw data from the scanotron has already been preprocessed when we inverted it.
inpath= "..\\data\\interim\\"+instr+"\\inverted\\pro\\" 
outpath = '..\\data\\interim\\'+instr+'\\combinedtimeseries\\'

# Call the clean_inverted function to prepare smps data. Give the input path, nbins, and outputh path.
df, outName, dLogDp=pt.clean_inverted(inpath=inpath, nbins=26, outpath=outpath)

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h3>3.1.2 Prepare Cleaned SMPS Data</h3>
</div>

In [15]:
# load scanotron data that was cleaned using pt.clean_inverted. Look inside the combinedtimeseries folder for the dates.
dates = '2020-03-16_2020-03-18'
instr = 'scanotron' 

dN, dNdLogDp_full = pt.load_scano_data(dates, instr)

This next step is mandatory. We want to average SMPS across the times that we were sampling INP. To do this, we create a dictionary where the keys are the day number or analysis number and the values are the INP sampling time. 

For example, here we have one INP sample that we collected from 11:25 on 2020-03-17 until 09:45 on 2020-03-18.  

In [16]:
in_time_dict = {    
    '17': ['2020-03-17 11:25','2020-03-18 09:45'],
    }

Next, we will loop through the dictionary we just created and extract the corresponding SMPS data.

In [17]:
# create empty smps dictionary
smps_dict = {}
# loop through each key in the in_time_dict and index those times from the dNdlogDp_full dataframe and save each of them as a dictionary in the smps dictionary
for day in in_time_dict:
    smps_dict[day] = dNdLogDp_full.loc[in_time_dict[day][0]:in_time_dict[day][1]]

The following piece of code is optional. We use it to remove outliers.

In [18]:
# loop through the dictionary and remove massive outliers
from scipy import stats
for day in in_time_dict:
    z = abs(stats.zscore(smps_dict[day].sum(axis=1)))
    smps_dict[day] = smps_dict[day][(z<1)]

Now we'll take our extracted SMPS data and average it and calculate standard deviation.

In [19]:
# create a daily standard deviation from the smps dictionary
smps_daily_std = {}
smps_daily_mean = {}

# create a daily mean dictionary from the smps dictionary
for day in smps_dict:
    smps_daily_mean[day] = smps_dict[day].mean()
    smps_daily_std[day] = smps_dict[day].std()

# turn them into dataframes
smps_daily_mean_df = pd.DataFrame(smps_daily_mean)
smps_daily_std_df = pd.DataFrame(smps_daily_std)

# clean it up a bit
smps_daily_mean_df.reset_index(inplace=True)
smps_daily_mean_df=smps_daily_mean_df.rename(columns={'index':'Dp'})
smps_daily_mean_df.set_index('Dp', inplace=True)

smps_daily_std_df.reset_index(inplace=True)
smps_daily_std_df=smps_daily_std_df.rename(columns={'index':'Dp'})
smps_daily_std_df.set_index('Dp', inplace=True)
smps_daily_mean_df.index=smps_daily_mean_df.index.astype(int)
smps_daily_std_df.index=smps_daily_std_df.index.astype(int)

Finally, we have a cleaned dataframe of the average particle size distibution across our INP sampling time. Each column corresponds to a specific INP sample and each row is the average particle count for a given diameter.

In [20]:
smps_daily_mean_df.head()

,17
Dp,
10,519.529412
12,496.356471
14,443.970588
16,479.923529
19,546.934118


We also have a dataframe for the standard deviation.

In [22]:
smps_daily_std_df.head()

,17
Dp,
10,308.922551
12,454.732329
14,273.922232
16,294.926888
19,365.612492


Now we will calculate the surface area. We pass it the two dataframes we just created, along with the number of bins.

In [24]:
dAdLogDp, dA_total, dN_total, dAdLogDp_std = pt.surface_area(smps_daily_mean_df, smps_daily_std_df, 25)

In [ ]:
pt.plot_number_dist()

Finally, we create plots by passing the data to the pt.plot_number_dist and pt.plot_surface_dist() functions.

 Note that the functions require the dataframes to  only contain the particle count (or surface area) data, where rows are size bins (as the index) and columns are the mean of daily (or other timespan) data. This means we have to drop a few columns. See below for explanation.

In [33]:
# Our smps dataframe has 3 columns that we don't need.
smps_daily_mean_df.head()

,17,Dp (m),Dp2 (m2),factorA
Dp,,,,
10,519.529412,1.000000e-08,1.000000e-16,3.141590e-16
12,496.356471,1.200000e-08,1.440000e-16,4.523890e-16
14,443.970588,1.400000e-08,1.960000e-16,6.157516e-16
16,479.923529,1.600000e-08,2.560000e-16,8.042470e-16
19,546.934118,1.900000e-08,3.610000e-16,1.134114e-15


In [35]:
# This is how the dataframe will need to look. We've dropped the three extra columns
smps_daily_mean_df.drop(columns=['Dp (m)', 'Dp2 (m2)', 'factorA']).head()

,17
Dp,
10,519.529412
12,496.356471
14,443.970588
16,479.923529
19,546.934118


In [25]:
fig3 = pt.plot_number_dist(smps_daily_mean_df.drop(columns=['Dp (m)', 'Dp2 (m2)', 'factorA']), smps_daily_std_df.drop(columns=['Dp (m)', 'Dp2 (m2)', 'factorA']))
fig4 = pt.plot_surface_dist(dAdLogDp, dAdLogDp_std)

In [28]:
fig3.update_layout(height=400, width=400)

In [30]:
fig4.update_layout(height=400, width=400)

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h3>3.1.3 Normalize INP Data </h3>
</div>

See the Pyce Tools manual for deeper explanation of the code below. Basically, we use create an INP object which consists of some INP data and corresponding biology data. We then use the surface area distribution we calculated above to calculate surface area normalized concentration.

In [52]:
some_bio_dataframe = pd.DataFrame(columns=['location'])
some_other_bio_dataframe = pd.DataFrame(columns=['location'])

inp_bubbler = pt.inp(inp_type='aerosol', inp_location='bubbler', cyto_location='n/a', cyto_data=some_bio_dataframe, uway_bio_data=some_other_bio_dataframe, inp_data=bubbler_df)

inp_bubbler.sa_normalize(dA_total)

done


In [54]:
# NOTE: If you want to compare to different literature values, you can alter the function in the source code.
fig2 = inp_bubbler.plot_ins_inp()
fig2.update_yaxes(type='log', title='INP per cm<sup>2</sup> of SSA Surface (D<sub>p</sub> = 10-5000nm)', exponentformat='power')

<div class="alert alert-block alert-info" style="outline-style: solid;">
<h2>3.2 Calculate Correlations </h2>
</div>


See the manual for detailed explanation on calculating correlations.

In [ ]:
inp_bubbler.inp.set_index('datetime',inplace=True)
inp_bubbler.correlations(temps=temps,process='UH', inp_units='inp_sa_normalized', size='sub')
inp_bubbler.correlations(temps=temps,process='H', inp_units='inp_sa_normalized', size='sub')

In [ ]:
temp = '-17.5'
inp_bubbler.results['UH'][temp]['corrs'].sort_values(by='p', ascending=True)[:10].style.set_caption(f"Correlations with Unheated Bubbler INP {temp}")